In [90]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='9046db90-cfdf-43f6-a674-1b0eb1f761b6', project_access_token='p-a2caa506ee09cf5e868182aedc417cade437e5ec')
pc = project.project_context


### testing box connection within watson studio and creating pandas dataframe from xls file uploaded in box

1. no need to establish connection in watson studio, yet to post box app config file as asset in watson studio <br>
2. Once the file is accessible, then a box client connection can be created from setting files <br>
3. Once the connection is done, then .content() instruction is used to download the file <br>
4. Lastly, read_excel() is used to write the pandas dataframe 

In [91]:
import pandas as pd
import numpy as np
import os, sys
from ibm_botocore.client import Config
import ibm_boto3

!pip install "boxsdk[jwt]"
from boxsdk import Client
from boxsdk import JWTAuth


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [92]:
credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-5c432b04-453d-486a-8d5b-8d2471bf03a9',
    'IBM_API_KEY_ID': 'n2gP77CYLRJZgkn9eSIKt_e0HW8bnWO6ZXikyxIigWMS',
    'ENDPOINT': 'https://s3.private.us.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'ibmeagitaly-donotdelete-pr-imp2elsawwkf1y',
    'FILE': '455328_eru3t25n_config.json'
}


In [93]:
cos = ibm_boto3.client(service_name='s3',
ibm_api_key_id=credentials['IBM_API_KEY_ID'],
ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
config=Config(signature_version='oauth'),
endpoint_url=credentials['ENDPOINT'])

In [94]:
from ibm_botocore.client import Config
import ibm_boto3
import types

def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print("File Downloaded")

In [95]:
!rm -rf /home/wsuser/work/*

In [96]:
cos.download_file(Bucket=credentials['BUCKET'],Key='455328_eru3t25n_config.json',Filename='/home/wsuser/work/455328_eru3t25n_config.json')

In [97]:
!pwd

/home/wsuser/work


In [98]:
!ls /home/wsuser/work

455328_eru3t25n_config.json


In [99]:
config = JWTAuth.from_settings_file('/home/wsuser/work/455328_eru3t25n_config.json')
client = Client(config)


myboxfile = client.file('817144087668').get()
print(myboxfile)
file_content = client.file('817144087668').content()

<Box File - 817144087668 (contracts.xlsx)>


In [100]:
df=pd.read_excel(file_content)
df.shape

(6, 11)

In [101]:
df.head()

,ID,Name,Sign date,Exp date,Type,Amount,Contact in Ecosystem,Distributor,Notes,IMT,Country
0,3ba62c3828fc4bd9b96227b9456f1868,UBICO,2020-12-31,2021-12-31,ESA,6662,Angelini,CGI,Possibile estensione 2021,Italy,Italy
1,f419656cfa0143adaab7f3d914e702de,DeepLab,2020-12-31,2021-12-31,ESA,10000,Angelini,CGI,Opportunità estensione a 25K,Italy,Italy
2,2c11b2867bf54260a02b9bb1bd06c191,Confirmo,2020-12-31,2021-12-31,CSA,13130,Roman,Dublin,opportunità blockchain,Italy,Italy
3,3f5a91a139604c1595e2131b371771a4,PatchAI,2020-03-19,2021-03-18,ESA,24242,Parisi,CGI,NaN,Italy,Italy
4,b99e6af10e96426e979049dea44e0e7e,Fortunito,2020-09-20,2021-03-18,CSA,8000,NaN,NaN,NaN,Italy,Italy


In [102]:
print(type(df['Sign date'][1]))


<class 'pandas._libs.tslibs.timestamps.Timestamp'>


### writing dataframe in excel and then posting it to box

In [103]:


from io import BytesIO

output = BytesIO()
writer=pd.ExcelWriter(output, engine='xlsxwriter')
df.to_excel(writer, index=False, sheet_name='sheet')

workbook = writer.book
worksheet1=writer.sheets['sheet']
header_format = workbook.add_format ({ 'bold' : True, 'bg_color': '2EECE6', 'border':1})
worksheet1.freeze_panes(1,0)

def get_col_widths(dataframe): 
    return [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

for i, width in enumerate(get_col_widths(df)):
    worksheet1.set_column(i, i, width)

workbook.close()
writer.save()
output.seek(0)
assetname="test.xlsx"


project.save_data(assetname, output, set_project_asset=True, overwrite=True)



{'file_name': 'test.xlsx',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmeagitaly-donotdelete-pr-imp2elsawwkf1y',
 'asset_id': 'f3a3fa46-139d-47f2-a3b0-e6dd392fed13'}

In [104]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-5c432b04-453d-486a-8d5b-8d2471bf03a9',
    'IBM_API_KEY_ID': 'n2gP77CYLRJZgkn9eSIKt_e0HW8bnWO6ZXikyxIigWMS',
    'ENDPOINT': 'https://s3.private.us.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'ibmeagitaly-donotdelete-pr-imp2elsawwkf1y',
    'FILE': 'test.xlsx'
}


In [105]:


cos = ibm_boto3.client(service_name='s3',
ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
config=Config(signature_version='oauth'),
endpoint_url=credentials_1['ENDPOINT'])



In [106]:
cos.download_file(Bucket=credentials_1['BUCKET'],Key='test.xlsx',Filename='/home/wsuser/work/test.xlsx')

In [107]:
#FIRST TIME
#folder_id="131084746758"
#new_file = client.folder(folder_id).upload('/home/wsuser/work/test.xlsx')
#print('File "{0}" uploaded to Box with file ID {1}'.format(new_file.name, new_file.id))

#SECONTIME TIME ON
file_id = '831822997599'
file_path = '/home/wsuser/work/test.xlsx'

updated_file = client.file(file_id).update_contents(file_path)
print('File "{0}" has been updated'.format(updated_file.name))

File "test.xlsx" has been updated
